In [5]:
get_ipython().magic('matplotlib inline')
get_ipython().magic('run /home/fenics/fenics-matplotlib.py')
from dolfin import *; 
from mshr import *; 
from IPython.display import display, clear_output; 
import time;
import logging; 
logging.getLogger('FFC').setLevel(logging.WARNING)

resolution=5
mesh = UnitSquareMesh(resolution, resolution)

def AllBoundary(x, on_boundary):
    return on_boundary

zero = Constant(0.0)

def aform(u, v):
    return inner(grad(u), grad(v))

def Lform(f, v):
    return f*v

### Primal problem
f = Expression("32.*x[0]*(1. - x[0])+32.*x[1]*(1. - x[1])", domain=mesh, degree=5)
ue = Expression("16.*x[0]*(1. - x[0])*x[1]*(1. - x[1])", domain=mesh, degree=5)

Qp = FunctionSpace(mesh,'CG',1);
bcp = DirichletBC(Qp, zero, AllBoundary)

u = TrialFunction(Qp);
v = TestFunction(Qp);

ap = aform(u, v)*dx 
Lp = Lform(f, v)*dx 

U = Function(Qp)
solve(ap == Lp, U, bcp)

### Dual problem
Qd = FunctionSpace(mesh, 'CG', 2);
psi = Constant(1.0)
bcd = DirichletBC(Qd, zero, AllBoundary)

w = TestFunction(Qd);
phi = TrialFunction(Qd);
ad = aform(w, phi)*dx 
Ld = Lform(psi, w)*dx
phi = Function(Qd)
solve(ad == Ld, phi, bcd)

e1=assemble((U-ue)*psi*dx)
e2=assemble((inner(grad(U),grad(phi))-f*phi)*dx)

print "e1: ", e1
print "e2: ", e2

e1:  -0.0480426666667
e2:  -0.0474366070504
